# Basic BVAS demo using GISAID S-gene data

In [7]:
import torch
import gzip
from bvas import BVASSelector, map_inference

### Load data

In [2]:
data = torch.load(gzip.GzipFile("../data/S_gene.pt.gz", "rb"), map_location='cpu')

In [3]:
# inspect data
for k, v in data.items():
    if hasattr(v, 'shape'):
        print(k, v.shape)
    elif isinstance(v, list):
        print(k, len(v))
    else:
        print(k, v)

Gamma torch.Size([415, 415])
Y torch.Size([415])
num_alleles 415
num_regions 74
mutations 415


### Instantiate BVASSelector object

In [4]:
selector = BVASSelector(data['Y'], 
                        data['Gamma'], 
                        data['mutations'], 
                        nu_eff=0.25,
                        S=5.0,
                        tau=100.0)

### Run BVAS MCMC-based inference

In [5]:
selector.run(T=5000, T_burnin=1000, seed=1)

  0%|          | 0/6000 [00:00<?, ?it/s]

### Inspect results

The results can be found in the `selector.summary` Pandas DataFrame.

In [6]:
print(selector.summary.iloc[:30][['PIP', 'Beta', 'BetaStd', 'Rank']])

              PIP      Beta   BetaStd  Rank
S:P681R  0.999999  0.441865  0.051502     1
S:R346K  0.999999  0.480412  0.058526     2
S:T478K  0.999999  0.482250  0.074330     3
S:L452R  0.999999  0.493581  0.060148     4
S:T19R   0.999999  0.556053  0.075309     5
S:E484K  0.999999  0.322367  0.033767     6
S:N440K  0.999999  0.490195  0.071955     7
S:N501Y  0.999138  0.286523  0.045726     8
S:T95I   0.998577  0.305510  0.085458     9
S:N969K  0.996153  0.432462  0.103096    10
S:Q954H  0.995751  0.430293  0.103179    11
S:G339D  0.995335  0.426014  0.087030    12
S:N679K  0.993701  0.397688  0.097671    13
S:N764K  0.983076  0.383790  0.108570    14
S:S375F  0.977092  0.367649  0.113546    15
S:S373P  0.976986  0.366454  0.111106    16
S:T859N  0.973080  0.263941  0.087984    17
S:S477N  0.932228  0.132079  0.055168    18
S:T716I  0.884272  0.257693  0.123831    19
S:Y145H  0.873942  0.194100  0.089103    20
S:H655Y  0.851670  0.256922  0.121269    21
S:P681H  0.698619  0.122920  0.0

## Let's compare to MAP inference

In [9]:
map_results = map_inference(data['Y'], data['Gamma'], data['mutations'], tau_reg=2048.0)

In [11]:
map_results.iloc[:30]

,Beta,BetaStd,Rank
S:T478K,0.479280,0.016720,1
S:L452R,0.403807,0.015633,2
S:T19R,0.401983,0.018214,3
S:P681R,0.385612,0.017929,4
S:N440K,0.255411,0.019505,5
S:R346K,0.232117,0.018944,6
S:T95I,0.226313,0.016852,7
S:N969K,0.223328,0.021557,8
S:Q954H,0.222891,0.021557,9
S:G339D,0.222523,0.021557,10


### Compare uncertainty estimates

We note that the MAP uncertainty estimates are much narrower than the 
corresponding BVAS uncertainty estimates. This is ultimatedly due to
the fact that BVAS considers multiple hypotheses about which alleles
are neutral and which are not.

In [16]:
# BVAS posterior standard deviation of selection coefficient for S:T478K 
selector.summary.loc['S:T478K'].BetaStd

0.07433029264211655

In [18]:
# MAP posterior standard deviation of selection coefficient for S:T478K 
map_results.loc['S:T478K'].BetaStd

0.016719751060009003

In [21]:
# compute ratio
selector.summary.loc['S:T478K'].BetaStd / map_results.loc['S:T478K'].BetaStd

4.44565785551095